In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              RandomTreesEmbedding, VotingClassifier)
from sklearn.linear_model import LogisticRegression
#датасет UCI ML Breast Cancer Wisconsin. 

In [2]:
X, y = load_breast_cancer(return_X_y=True)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## 1) Реализовать функцию нахождения самого частого элемента в массиве (голосование)


In [4]:
def get_most_frequent_value(sequence):
    unique, counts = np.unique(sequence, return_counts=True)
    d = dict(zip(unique, counts))
    i = list(d.values()).index(max(counts))

    return list(d.keys())[i]

In [5]:
get_most_frequent_value([1, 2, 3, 3, 3, 4, 4])

3

## 2) Реализовать функцию, выполняющую бутстреп. Ваш алгоритм должен сохранить размерность входного параметра values! В данном случае, функция должна вернуть одномерный массив. 

In [6]:
def bootstrap(values, new_dataset_size):
    """
    Creates a new dataset from the old one using bootstrap.
    
    :arg values: np.array of shape (n_objects, n_features), input objects
    :arg new_dataset_size: int, number of elements in the resulting array
    
    :return: bootstraped_dataset: np.array of shape (new_dataset_size, n_features)
    """
    # TODO your code here
    Result_arr = np.empty((0,values.shape[1]), int)
    for i in range(new_dataset_size):
        Result_arr = np.append(Result_arr, values[np.random.choice(range(0,values.shape[0]),1)], axis = 0)
    return Result_arr

In [7]:
matr = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [8]:
bootstrap(matr, 5)

array([[7, 8, 9],
       [1, 2, 3],
       [1, 2, 3],
       [7, 8, 9],
       [7, 8, 9]])

## 3 Реализуйте функцию для подсчёта произвольной метрики качества классификации из Scikit-Learn (например, f1_score) в режиме out-ot-bag. Для этого воспользуйтесь аттрибутом oob_decision_function_, который появляется в обученном случайном лесе. Он содержит предсказания для всех объектов обучающей выборки — для каждого объекта они получены агрегацией ответов тех деревьев, которые не использовали во время обучения этот конкретный объект.

In [9]:
# Write-up
def estimate_oob_metric(forest, metric):
    """
    Computes any classification metric in the out-of-bag mode.
        :arg forest: Scikit-Learn ensemble model
        :arg metric: callable with two arguments that returns a float
        
        :return oob_metric_value: float
    """
    return forest.oob_score_

In [10]:
forest = RandomForestClassifier(n_estimators=100, max_depth=8, criterion='entropy', oob_score=True, random_state=42)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
f1 = f1_score(y_train, y_pred)

estimate_oob_metric(forest, f1)

0.967032967032967

## Задачи с проверкой ответа

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.  

Случайный лес — стохастический алгоритм, а значит при каждом запуске может выдавать различный результат. В дальнейших задачах __необходимо__ выставить случайность: добавить параметр `random_state=8` как аргумент при создании ансамблей.

In [11]:
def compute_metric(clf, X_train=X_train, y_train=y_train,
                   X_test=X_test, y_test=y_test):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [12]:
forest = RandomForestClassifier(random_state=8)
compute_metric(forest)

0.965035

## 4 Напишите индекс самого важного признака, по мнению случайного леса.

In [13]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
print('Важность признаков:', forest.feature_importances_)
max_value = max(forest.feature_importances_)
print('Индекс самого важного признака:', list(forest.feature_importances_).index(max_value))

Важность признаков: [0.04870337 0.01359088 0.05326975 0.04755501 0.00728533 0.01394433
 0.06800084 0.10620999 0.00377029 0.00388577 0.02013892 0.00472399
 0.01130301 0.02240696 0.00427091 0.00525322 0.00938583 0.00351326
 0.00401842 0.00532146 0.07798688 0.02174901 0.06711483 0.15389236
 0.01064421 0.02026604 0.0318016  0.14466327 0.01012018 0.00521012]
Индекс самого важного признака: 23


## 5 Увеличьте количество деревьев в случайном лесе до 100. Как изменилось качество? Укажите его. 

In [14]:
forest = RandomForestClassifier(random_state=8, n_estimators=100)
compute_metric(forest)

0.965035

## 6 Выставите количество деревьев в лесе равным 100 и включите режим подсчёта out-of-bag score. Укажите полученное значение метрики accuracy

In [15]:
def calc_accuracy(y_true, y_pred):
    P = np.sum(y_true == 1)
    N = np.sum(y_true == 0)
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    acc = (TP + TN) / (P + N)
    
    return acc

In [16]:
forest = RandomForestClassifier(random_state=8, n_estimators=100, oob_score=True)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
acc = calc_accuracy(y_train, y_pred)
print('acc = {}%'.format(acc * 100))

acc = 100.0%


## 7 Выключите режим random subspaces в случайном лесе. Укажите полученное качество.

In [17]:
forest = RandomForestClassifier(n_estimators=100, oob_score=True)
compute_metric(forest)

0.972222

## 8 Выключите режим бутстрапа и переобучите случайный лес. Укажите полученное качество.

In [18]:
forest = RandomForestClassifier(n_estimators=100, bootstrap=False)
compute_metric(forest)

0.965035

## 9 Какая средняя глубина у деревьев, которые получаются в случайном лесе со стандартными параметрами при его обучении на этой обучающей выборке?

In [19]:
sum_depth = 0
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
for i in range(forest.n_estimators):
    sum_depth += forest.estimators_[0].tree_.max_depth
print('Средняя глубина деревьев леса =', sum_depth/forest.n_estimators)

Средняя глубина деревьев леса = 7.0
